In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime
urlfile="https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv"
costs = pd.read_csv(urlfile)
costs["Symbol"]
#

0       MMM
1       AOS
2       ABT
3      ABBV
4      ABMD
       ... 
500     YUM
501    ZBRA
502     ZBH
503    ZION
504     ZTS
Name: Symbol, Length: 505, dtype: object

In [2]:
pp= []
for i in costs["Symbol"]:
    i = (str(i).split(' ')[0])
    pp.append(i)
lst = pd.DataFrame(pp)
lst = lst.rename(columns = {0:'ticker'})

In [4]:
lst

,ticker
0,MMM
1,AOS
2,ABT
3,ABBV
4,ABMD
...,...
500,YUM
501,ZBRA
502,ZBH
503,ZION


In [12]:
def yf_downloader(pp, n = 10, start_date = '2010-12-31', end_date = 'today', freq_weekly = False):
    ppp =[]
    htop = {}
    lst = pp.sample(n)
    if end_date == 'today':
        from datetime import datetime
        end_date = datetime.today().strftime('%Y-%m-%d')
    for i in list(lst['ticker']):
        print(i)
        
        #end_date = datetime.today().strftime('%Y-%m-%d')
        
    
        xxx = yf.download(str(i), start = start_date, end = end_date, interval = '1d')['Close']
        
        #xxx = pd.date_range(xxx.index.min(), xxx.index.max()).to_frame().merge(xxx, how='left', right_index = True, left_index = True)
        
        #xxx.drop(0, axis = 1, inplace = True)
        
        xxx= pd.DataFrame(xxx)
        
        xxx.rename(columns = {'Close':'priceclose'}, inplace = True)
        
        if freq_weekly:
            xxx = xxx[xxx.index.dayofweek == 4]
        
        
        xxx['companyid'] = i
        xxx.index.name = 'pricingdate'
        ppp.append(xxx)
        
        
    port = pd.concat(ppp, axis = 1)
    portfolio_price = port['priceclose'].sum(axis = 1)
    a = pd.DataFrame(portfolio_price)
    
        
    return     a

In [15]:
randomgen = yf_downloader(lst, n=10, start_date = '2000-12-31', end_date = 'today', freq_weekly = False)
randomgen = randomgen.rename(columns = {0: 'priceclose'})
randomgen

VRTX
[*********************100%***********************]  1 of 1 completed
HBAN
[*********************100%***********************]  1 of 1 completed
WY
[*********************100%***********************]  1 of 1 completed
AVB
[*********************100%***********************]  1 of 1 completed
FFIV
[*********************100%***********************]  1 of 1 completed
NOW
[*********************100%***********************]  1 of 1 completed
LMT
[*********************100%***********************]  1 of 1 completed
PGR
[*********************100%***********************]  1 of 1 completed
EA
[*********************100%***********************]  1 of 1 completed
SO
[*********************100%***********************]  1 of 1 completed


,priceclose
pricingdate,
2001-01-02,265.447356
2001-01-03,268.633857
2001-01-04,266.142144
2001-01-05,253.156482
2001-01-08,245.341326
...,...
2022-05-24,1871.590002
2022-05-25,1913.690022
2022-05-26,1923.819968


In [16]:
randomgen['r1'] = randomgen.pct_change()
randomgen

,priceclose,r1
pricingdate,,
2001-01-02,265.447356,NaN
2001-01-03,268.633857,0.012004
2001-01-04,266.142144,-0.009275
2001-01-05,253.156482,-0.048792
2001-01-08,245.341326,-0.030871
...,...,...
2022-05-24,1871.590002,0.002630
2022-05-25,1913.690022,0.022494
2022-05-26,1923.819968,0.005293


In [17]:
randomgen['signal'] = 1*(randomgen['r1']>0)
randomgen

,priceclose,r1,signal
pricingdate,,,
2001-01-02,265.447356,NaN,0
2001-01-03,268.633857,0.012004,1
2001-01-04,266.142144,-0.009275,0
2001-01-05,253.156482,-0.048792,0
2001-01-08,245.341326,-0.030871,0
...,...,...,...
2022-05-24,1871.590002,0.002630,1
2022-05-25,1913.690022,0.022494,1
2022-05-26,1923.819968,0.005293,1


In [18]:
randomgen = randomgen.dropna()
randomgen

,priceclose,r1,signal
pricingdate,,,
2001-01-03,268.633857,0.012004,1
2001-01-04,266.142144,-0.009275,0
2001-01-05,253.156482,-0.048792,0
2001-01-08,245.341326,-0.030871,0
2001-01-09,247.053257,0.006978,1
...,...,...,...
2022-05-24,1871.590002,0.002630,1
2022-05-25,1913.690022,0.022494,1
2022-05-26,1923.819968,0.005293,1


In [20]:
randomgen.to_csv('HMM_randomportfolio.csv')